<a href="https://colab.research.google.com/github/antonum/Redis-Workshops/blob/main/03-Advanced_RedisSearch/03-Advanced_RedisSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced RediSearch

![Redis](https://redis.com/wp-content/themes/wpx/assets/images/logo-redis.svg?auto=webp&quality=85,75&width=120)

This notebook is an adapted and simplified version of the RedisInsight QuickGuide "Working with Hashes".

For the full exterience we'd recommend installing Redis Insight and going through tutorial there.

https://redis.com/redis-enterprise/redis-insight/

In [1]:
# Install the requirements
!pip install -q redis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 11.1 MB/s eta 0:00:00


In [2]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes


deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb focal main
Starting redis-stack-server, database path /var/lib/redis-stack


In [3]:
import redis
import os


In [4]:
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#Replace values above with your own if using Redis Cloud instance
#REDIS_HOST="redis-12110.c82.us-east-1-2.ec2.cloud.redislabs.com"
#REDIS_PORT=12110
#REDIS_PASSWORD="pobhBJP7Psicp2gV0iqa2ZOc1XXXXXX"

#shortcut for redis-cli $REDIS_CONN command
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

In [5]:
r = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)
r.ping()

True

## Redis Hashes

Hash is a fundamental Redis datatype.


See full list of Redis HASH commands here: https://redis.io/commands/?group=hash




In [6]:
schools = [
    {"name":"Hall School",
     "description":"Spanning 10 states, this school award-winning curriculum includes a comprehensive reading system (from letter recognition and phonics to reading full-length books), as well as math, science, social studies, and even  philosophy.",
     "class":"independent",
     "type":"traditional",
     "address_city":"London",
     "address_street":"Manor Street",
     "students":342,
     "location":"51.445417, -0.258352"
     },
    {"name":"Garden School",
     "description":"Garden School is a new and innovative outdoor teaching and learning experience, offering rich and varied activities in a natural environment to children and families.",
     "class":"state","type":"forest; montessori",
     "address_city":"London",
     "address_street":"Gordon Street",
     "students":1452,
     "location":"51.402926, -0.321523",
     },

    {"name":"Gillford School",
     "description":"Gillford School is an inclusive learning centre welcoming people from all walks of life, here invited to step into their role as regenerative agents, creating new pathways into the future and inciting an international movement of cultural, land, and social transformation.",
     "class":"private",
     "type":"democratic; waldorf",
     "address_city":"Goudhurst",
     "address_street":"Goudhurst",
     "students":721,
     "location":"51.112685, 0.451076"
     },

     {
     "name":"Forest School",
     "description":"The philosophy behind Forest School is based upon the desire to provide young children with an education that encourages appreciation of the wide world in nature while achieving independence, confidence and high self-esteem. ",
     "class":"independent",
     "type":"forest; montessori; democratic",
     "address_city":"Oxford",
     "address_street":"Trident Street",
     "students":1200,
     "location":"51.781756, -1.123196"
     }
    ]
#load data in Redis as JSON
for id,school in enumerate(schools):
    #print(school)
    r.hset(f"school:{id}", mapping = school)

In [7]:
!redis-cli $REDIS_CONN keys 'school:*'
!echo
!redis-cli $REDIS_CONN HGETALL school:1



1) "school:0"
2) "school:3"
3) "school:2"
4) "school:1"

 1) "location"
 2) "51.402926, -0.321523"
 3) "address_street"
 4) "Gordon Street"
 5) "address_city"
 6) "London"
 7) "students"
 8) "1452"
 9) "class"
10) "state"
11) "name"
12) "Garden School"
13) "description"
14) "Garden School is a new and innovative outdoor teaching and learning experience, offering rich and varied activities in a natural environment to children and families."
15) "type"
16) "forest; montessori"


In [8]:
#!redis-cli $REDIS_CONN flushdb

## RediSearch

RediSearch adds the ability to query data in your HASH or JSON data structures, essentially turning Redis into the docuemnt database.

With RediSearch you declare indices once and then every database object matching the prefix, defined in the index would be automatically and in real time added to the index.

For the full list of RediSearch commands see: https://redis.io/commands/?group=search

Python documentation: https://redis-py.readthedocs.io/en/stable/redismodules.html#redisearch-commands

In [9]:
from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    GeoField,
    VectorField
)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query
from redis.commands.search.aggregation import AggregateRequest
from redis.commands.search import reducers
schema = (
    TextField("name", as_name="name"),
    TextField("description", as_name="description"),
    TagField("address_city", as_name="city"),
    NumericField("students", as_name="students"),
    GeoField("location", as_name="location")
    )
r.ft("idx:schools").create_index(schema,
                    definition=IndexDefinition(prefix=["school:"],
                    index_type=IndexType.HASH)
                    )

b'OK'

In [10]:
#Retreive index information
r.ft("idx:schools").info()


{'index_name': 'idx:schools',
 'index_options': [],
 'index_definition': [b'key_type',
  b'HASH',
  b'prefixes',
  [b'school:'],
  b'default_score',
  b'1'],
 'attributes': [[b'identifier',
   b'name',
   b'attribute',
   b'name',
   b'type',
   b'TEXT',
   b'WEIGHT',
   b'1'],
  [b'identifier',
   b'description',
   b'attribute',
   b'description',
   b'type',
   b'TEXT',
   b'WEIGHT',
   b'1'],
  [b'identifier',
   b'address_city',
   b'attribute',
   b'city',
   b'type',
   b'TAG',
   b'SEPARATOR',
   b','],
  [b'identifier', b'students', b'attribute', b'students', b'type', b'NUMERIC'],
  [b'identifier', b'location', b'attribute', b'location', b'type', b'GEO']],
 'num_docs': '4',
 'max_doc_id': '4',
 'num_terms': '130',
 'num_records': '154',
 'inverted_sz_mb': '0.00087451934814453125',
 'vector_index_sz_mb': '0',
 'total_inverted_index_blocks': '137',
 'offset_vectors_sz_mb': '0.00014400482177734375',
 'doc_table_size_mb': '0.00028228759765625',
 'sortable_values_size_mb': '0',
 'k

In [11]:
import pandas as pd
def display_ft(res):
  if res.total==0:
    print("No matches found")
  else:
    res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
    display(res_df)

In [12]:
#return the entire document
res=r.ft("idx:schools").search("nature")
display_ft(res)

,id,name,address_street,address_city,type,location,description,students,class
0,school:3,Forest School,Trident Street,Oxford,forest; montessori; democratic,"51.781756, -1.123196",The philosophy behind Forest School is based u...,1200,independent
1,school:1,Garden School,Gordon Street,London,forest; montessori,"51.402926, -0.321523",Garden School is a new and innovative outdoor ...,1452,state


In [13]:
#return selected fields only
query=Query("nature") \
   .return_field("address_city", as_field="city") \
   .return_field("name", as_field="name")
res=r.ft("idx:schools").search(query)
display_ft(res)


,id,city,name
0,school:3,Oxford,Forest School
1,school:1,London,Garden School


In [14]:
#Geolocation search
query = Query('@location:[51.3 0.32 30 km]')
res = r.ft('idx:schools').search(query)
display_ft(res)

,id,name,address_street,address_city,type,location,description,students,class
0,school:2,Gillford School,Goudhurst,Goudhurst,democratic; waldorf,"51.112685, 0.451076",Gillford School is an inclusive learning centr...,721,private


In [15]:
#Multi-field query
query=Query('@city:{London} @students:[0, 10000]') \
   .return_field("address_city", as_field="city") \
   .return_field("name", as_field="name") \
   .return_field("students", as_field="students")
res=r.ft("idx:schools").search(query)
display_ft(res)


,id,city,name,students
0,school:0,London,Hall School,342
1,school:1,London,Garden School,1452


## Aggregations
Aggregations are a way to process the results of a search query, group, sort and transform them - and extract analytic insights from them. Much like aggregation queries in other databases and search engines, they can be used to create analytics reports, or perform Faceted Search style queries.

For example, we can group schools by city and count schools per group, giving us the number of schools per city. Or we could group by school class (independent/state) and see the average number of students per group.

In [22]:
#Perform aggregation by city and count number of records
request = AggregateRequest(f'*').group_by('@city', reducers.count().alias('count'))
res = r.ft("idx:schools").aggregate(request)
res.rows


[[b'city', b'London', b'count', b'2'],
 [b'city', b'Oxford', b'count', b'1'],
 [b'city', b'Goudhurst', b'count', b'1']]